In [1]:
import sys
sys.path.append("../venv/lib/python3.7/site-packages/")
from bokeh.io import output_file, show, save
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool
import json
from IPython.display import IFrame
import math
from ast import literal_eval

In [2]:
#https://towardsdatascience.com/
#exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
origin = merc('(41.795,-87.60)') # where are you (lat, long) ?

In [4]:
x_coords = [origin[0]]
y_coords = [origin[1]]

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

hover = HoverTool(tooltips=[
    ("station", "@stationname"),
    ("ridership","@ridership")
    
])

#source = ColumnDataSource(data=dict(
#                        x=list(Merged['coords_x']), 
#                        y=list(Merged['coords_y']),
#                        ridership=list(Merged['monthtotal']),
#                        sizes=list(Merged['circle_sizes']),
#                        stationname=list(Merged['STATION_NAME'])))

p = figure(x_range=(-18780000, 18000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator", 
           plot_width = 980, plot_height = 500, title = 'TCP/UDP End Points')
           #tools=[hover, '','wheel_zoom','save']
    
p.add_tile(tile_provider)

p.circle(x=x_coords, y=y_coords, size=10, color="green")

p.multi_line(xs=[[x_coords[0], x_coords[0]+2000]], 
             ys=[[y_coords[0], y_coords[0]+1000]],
             color=['red'])


p.multi_line(xs=[[x_coords[0], x_coords[0]+500]], 
             ys=[[y_coords[0], y_coords[0]+1000]],
             color=['green'])

output_file("tile1.html")
save(p)
IFrame(src='./tile1.html', width=1024, height=500)